In [ ]:
import random
import pandas as pd

In [ ]:
class Universitas:
    total_mahasiswa = 0
    anggaran = 0

    def __init__(self, nama_kampus, prodi):
        self.nama = nama_kampus
        self.prodi = prodi

    def hitung_dosen_kampus(self):
        total = 0
        for program_studi in self.prodi:
            total += program_studi.dosen
        return total
    
    def mahasiswa_lulus(self):
        for program_studi in self.prodi:
            program_studi.angkatan.pop(0)

In [ ]:
class Prodi:
    def __init__(self, nama, ukt, kode):
        self.nama = nama
        self.kode = kode
        self.ukt = ukt
        self.angkatan = []
        self.dosen = 0

    def tambah_angkatan_prodi(self, angkatan):
        self.angkatan.append(angkatan)

    def hitung_mahasiswa_prodi(self):
        total = 0
        for tiap_angkatan in self.angkatan:
            total += tiap_angkatan.hitung_mahasiswa_angkatan()
        return total

In [ ]:
class Angkatan:
    def __init__(self, tahun):
        self.tahun = tahun
        self.mahasiswa = []
        self.kelas = []

    def tambah_mahasiswa_angkatan(self, mahasiswa):
        self.mahasiswa.append(mahasiswa)

    def hitung_mahasiswa_angkatan(self):
        return len(self.mahasiswa)

In [ ]:
class Mahasiswa:
    def __init__(self,NIM):
        self.NIM = NIM

In [ ]:
class Kelas:
    def __init__(self,nama_kelas):
        self.nama_kelas = nama_kelas
        self.mahasiswa = []
    def tambah_mahasiswa_kelas(self,mahasiswa):
        self.mahasiswa.append(mahasiswa)
    def hitung_mahasiswa_kelas(self):
        return len(self.mahasiswa)

In [ ]:
prodi = [Prodi("Informatika",7000000,"IF"),
         Prodi("Sistem Informasi",8000000,"SI"),
         Prodi("Teknologi Indormasi",8000000,"TI"),
         Prodi("Rekayasa Perangkat Lunak",8500000,"RPL"),
         Prodi("Sains Data",7000000,"DS")]

In [ ]:
TelkomMadura = Universitas("Telkom Madura",prodi)

In [ ]:
def proses_mendaftar(NIM,tahun_angkatan):
    # MEMILIH PRODI
    prodi_terpilih = random.randint(0,4)
    mahasiswa_baru = Mahasiswa(NIM)
    prodi[prodi_terpilih].angkatan[tahun_angkatan].mahasiswa.append(mahasiswa_baru)
    
    # PEMBAYARAN
    BiayaUP3 = 8000000
    BiayaSDP = 12150000
    BiayaUKT = prodi[prodi_terpilih].ukt
    Total = BiayaSDP + BiayaUKT + BiayaUP3
    return Total

In [ ]:
def membuat_kelas(tahun_angkatan,tahun):
    for program_studi in prodi:
        total_mahasiswa = program_studi.angkatan[tahun_angkatan].hitung_mahasiswa_angkatan()
        total_kelas = total_mahasiswa // 30 + 1
        kode_kelas = 0

        total_kelas_counter = 0
        start = 0
        while total_kelas_counter < total_kelas and total_mahasiswa >= 0:
            program_studi.angkatan[tahun_angkatan].kelas.append(Kelas(f"{program_studi.kode}-{tahun}-{kode_kelas + 1}")) # menambahkan kelas baru
            for mahasiswa in program_studi.angkatan[tahun_angkatan].mahasiswa[start:]: # looping ke mahasiswa di angkatan tertentu
                program_studi.angkatan[tahun_angkatan].kelas[kode_kelas].mahasiswa.append(mahasiswa) # menambahkan mahasiswa ke kelas
                total_mahasiswa = total_mahasiswa - 1
                start += 1
                if program_studi.angkatan[tahun_angkatan].kelas[kode_kelas].hitung_mahasiswa_kelas() == 30: # jika mahasiswa per kelas sudah 30 maka break dan buat kelas baru
                    break
            kode_kelas += 1
            total_kelas_counter += 1

        # for _ in range(total_kelas):
        #     program_studi.angkatan[tahun_angkatan].kelas.append(Kelas(f"{program_studi.kode}-{tahun_angkatan}-{kode_kelas + 1}")) # menambahkan kelas baru
        #     for mahasiswa in program_studi.angkatan[tahun_angkatan].mahasiswa: # looping ke mahasiswa di angkatan tertentu
        #         program_studi.angkatan[tahun_angkatan].kelas[kode_kelas].mahasiswa.append(mahasiswa) # menambahkan mahasiswa ke kelas
        #         total_mahasiswa = total_mahasiswa - 1
        #         if program_studi.angkatan[tahun_angkatan].kelas[kode_kelas].hitung_mahasiswa_kelas() == 30: # jika mahasiswa per kelas sudah 30 maka break dan buat kelas baru
        #             break
        #     kode_kelas += 1
            
        # if total_mahasiswa - total_kelas * 30 != 0:
        #     sisa_mahasiswa = total_mahasiswa - total_mahasiswa * 30 
        #     for sisa in range(total_mahasiswa-sisa_mahasiswa,total_mahasiswa): # looping dari mahasiswa yang belum dapat kelas
        #         program_studi.angkatan[tahun_angkatan].kelas.append(Kelas(f"{program_studi.kode}-{tahun_angkatan}-{kode_kelas + 1}")) # menambahkan kelas baru
        #         program_studi.angkatan[tahun_angkatan].kelas[kode_kelas].mahasiswa.append(program_studi.angkatan[tahun_angkatan].mahasiswa[sisa]) # menambahkan mahasiswa sisa ke kelas baru
        #         if program_studi.angkatan[tahun_angkatan].kelas[kode_kelas].hitung_mahasiswa_kelas() == 30: # jika mahasiswa per kelas sudah 30 maka break dan buat kelas baru
        #             break
        #     kode_kelas += 1
        # KEBUTUHAN DOSEN
        hitung_kebutuhan_dosen(program_studi)

In [ ]:
def hitung_kebutuhan_dosen(program_studi):
    total_mahasiswa = program_studi.hitung_mahasiswa_prodi()
    kebutuhan_dosen = total_mahasiswa // 60
    program_studi.dosen = kebutuhan_dosen

In [ ]:
def pengeluaran_kampus(bulan):
    gaji_dosen = 7000000 * TelkomMadura.hitung_dosen_kampus()
    pengeluaran = 0.7 * TelkomMadura.anggaran / 12  + gaji_dosen
    if bulan == 12:
        pengeluaran += 200000000 # biaya gedung
    return pengeluaran

In [ ]:
def hitung_mahasiswa(tahun_angkatan):
    total = 0
    for program_studi in prodi:
        total += program_studi.angkatan[tahun_angkatan].hitung_mahasiswa_angkatan()
    return total

In [ ]:
def lihat_kelas():
    # for program_studi in prodi:
    #     for angkatan in program_studi.angkatan:
    #         print(f"{program_studi.nama} ({angkatan.tahun}) :")
    #         for kelas in angkatan.kelas:
    #             print(f"{kelas.nama_kelas} : {[mahasiswa.NIM for mahasiswa in kelas.mahasiswa]}")
    #     print("\n")
    for angkatan in prodi[0].angkatan:
        print(f"{angkatan.tahun} = {[kelas.nama_kelas for kelas in angkatan.kelas]}")

In [ ]:
def bayar_ukt(self):
    pemasukan = 0
    for program_studi in prodi:
        pemasukan += program_studi.ukt * program_studi.hitung_mahasiswa_prodi()
    return pemasukan

In [ ]:
def rekap_mahasiswa_prodi(tahun):
    mahasiswa = []
    for program_studi in prodi:
        mahasiswa.append(program_studi.angkatan[tahun].hitung_mahasiswa_angkatan())
    return mahasiswa

In [ ]:
def lihat_angkatan():
    print([i.tahun for i in prodi[0].angkatan])

In [ ]:
def simulasi_kampus(tahun_start,tahun_terakhir):
    angkatan = tahun_start
    growth_factor = 1.02 # kenaikan atau penurunan mahasiswa calon pendaftar
    mahasiswa = random.randint(1,100) # kemungkinan mahasiswa calon pendaftar
    nim = 1 # NIM atau ID mahasiswa
    cashflow = []
    data_mahasiswa = []
    # SIMULASI
    for tahun in range(tahun_terakhir - tahun_start): 
        # Membuat Angkatan Baru di Tiap Prodi
        for program_studi in prodi:
            program_studi.tambah_angkatan_prodi(Angkatan(tahun))
        for bulan in range(1,13):
            pemasukan = 0
            if(bulan == 2 or bulan == 9):
                pemasukan += bayar_ukt(prodi)
            # PENDAFTARAN   
            for _ in range(30): 
                if random.random() < 0.5: # Kemungkinan kenaikan calon pendaftar
                    mahasiswa +=  int(mahasiswa * random.uniform(0.05, 0.5) * growth_factor)
                else:
                    mahasiswa -= int(mahasiswa * random.uniform(0.05, 0.5) * growth_factor)
                # Mahasiswa Mendaftar
                for _ in range(mahasiswa):
                    if random.random() < 0.5:
                        pemasukan += proses_mendaftar(nim,-1)
                        nim += 1
                    if hitung_mahasiswa(-1) == 600:
                        break
            
            # PEMASUKAN TIAP BULAN
            TelkomMadura.anggaran += pemasukan
        
            # PENGELUARAN TIAP BULAN
            pengeluaran = pengeluaran_kampus(bulan)
            TelkomMadura.anggaran -= pengeluaran
            cashflow.append([angkatan,bulan,pemasukan,pengeluaran,TelkomMadura.anggaran])
            
        if tahun > 3:
            TelkomMadura.mahasiswa_lulus()
        
        # PEMBAGIAN KELAS
        membuat_kelas(-1,tahun)
        data_mahasiswa.append(rekap_mahasiswa_prodi(-1))
        # lihat_angkatan()
        lihat_kelas()
        print("\n")
        
        angkatan += 1
            
    return [cashflow,data_mahasiswa]

In [ ]:
result = simulasi_kampus(2021,2050)
result

In [ ]:
columns = ['tahun', 'bulan', 'pemasukan', 'pengeluaran', 'anggaran']
df = pd.DataFrame(result[0], columns=columns)
dfeachYear = []

tahuncolumns = df.tahun.unique().tolist()

for i in tahuncolumns:
    dfeachYear.append(df.loc[df.tahun == i])

In [ ]:
dfeachYear

In [ ]:
columns = ['Informatika', 'Sistem Informasi', 'Teknologi Informasi', 'Rekayasa Perangkat Lunak', 'Sains Data']
df = pd.DataFrame(result[1], columns=columns)
df

In [ ]:
lihat_kelas()